In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam

from keras.optimizers.legacy import Adam as LegacyAdam

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pickle as pkl
from src.utils.data_transform import *
from src.utils.preprocessing import load_split_data
import pandas as pd
import os 
import pickle 
import json
from src.utils.data_io import save_data

In [43]:
def load_data(test_subject, subject_to_indices):
    training_data, testing_data = [], []
    training_labels, testing_labels = [], []

    # Load data by sessions based on subject_to_indices mapping
    for subject, sessions in subject_to_indices.items():
        subject_data, subject_labels = [], []
        for session_id in sessions:
            session_data, session_labels = load_session_data(f"../data/ProcessedSubjects/MajorityLabel/sessions/grav_med_meanstd3/session_{session_id}.pkl")
            subject_data.append(session_data)
            subject_labels.append(session_labels)
        
        # Aggregate data for each subject
        subject_data = np.concatenate(subject_data, axis=0)
        subject_labels = np.concatenate(subject_labels, axis=0)
        
        # Distribute data into training or testing based on subject ID
        if str(subject) == str(test_subject):
            testing_data.append(subject_data)
            testing_labels.append(subject_labels)
        else:
            training_data.append(subject_data)
            training_labels.append(subject_labels)

    # Combine all training and testing data and labels
    training_data = np.concatenate(training_data, axis=0)
    training_labels = np.concatenate(training_labels, axis=0)
    testing_data = np.concatenate(testing_data, axis=0)
    testing_labels = np.concatenate(testing_labels, axis=0)

    return training_data, training_labels, testing_data, testing_labels

In [44]:
def load_session_data(path):
    data = pd.read_pickle(path)
    signal_data = np.array([item[0] for item in data])
    signal_data = signal_data[:, :, 1:]  # Exclude the timestamps column
    label_data = np.array([item[1] for item in data])
    return signal_data, label_data

In [45]:
def build_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))  # Assuming 5 classes for the output layer
    # optimizer = Adam(learning_rate=1e-3)
    optimizer = LegacyAdam(learning_rate=1e-3)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [46]:
with open("../data/dataset-info-json/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)

subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}

In [47]:
for test_subject in subject_to_indices.keys():
    # Load the data
    print(f"Training without {test_subject}")
    results = []
    accuracy = []
    loss = []
    model = build_model(input_shape=(20,6))
    train_data, train_labels, test_data, test_labels = load_data(test_subject, subject_to_indices)
    history = model.fit(train_data, train_labels, epochs=32, batch_size=64)
    results.append(model.evaluate(test_data, test_labels))
    accuracy.append(history.history['accuracy'])
    loss.append(history.history['loss'])
    model.save(f"../models/full_loso/majority_label/processed/mean_std_3/model_{test_subject}.keras")
    
    training_info_path = "../models/full_loso/majority_label/processed/mean_std_3/training_info/" 
    os.makedirs(training_info_path, exist_ok=True)
    save_data(results, training_info_path, f"results_{test_subject}")
    save_data(accuracy, training_info_path, f"accuracy_{test_subject}")
    save_data(loss, training_info_path, f"loss_{test_subject}")


Training without 1
Epoch 1/32
882/882 [==============================] - 11s 12ms/step - loss: 0.7561 - accuracy: 0.7377
Epoch 2/32
882/882 [==============================] - 11s 12ms/step - loss: 0.6544 - accuracy: 0.7739
Epoch 3/32
882/882 [==============================] - 11s 12ms/step - loss: 0.6315 - accuracy: 0.7818
Epoch 4/32
882/882 [==============================] - 11s 12ms/step - loss: 0.6167 - accuracy: 0.7886
Epoch 5/32
882/882 [==============================] - 11s 12ms/step - loss: 0.6024 - accuracy: 0.7929
Epoch 6/32
882/882 [==============================] - 11s 12ms/step - loss: 0.5925 - accuracy: 0.7944
Epoch 7/32
882/882 [==============================] - 11s 12ms/step - loss: 0.5862 - accuracy: 0.7975
Epoch 8/32
882/882 [==============================] - 11s 12ms/step - loss: 0.5772 - accuracy: 0.8006
Epoch 9/32
882/882 [==============================] - 11s 13ms/step - loss: 0.5718 - accuracy: 0.8007
Epoch 10/32
882/882 [==============================] - 11s 12ms

In [48]:
# history.history['accuracy']
tot_acc = []
tot_res = []
tot_loss = []
info_path = "../models/full_loso/majority_label/processed/mean_std_3/training_info"
for i in range(1,13):
    with open(f"{info_path}/accuracy_{i}.pkl", "rb") as a:
        tot_acc.append(pickle.load(a))
    with open(f"{info_path}/results_{i}.pkl", "rb") as r:
        tot_res.append(pickle.load(r))
    with open(f"{info_path}/loss_{i}.pkl", "rb") as l:
        tot_loss.append(pickle.load(l))

tot_res = np.concatenate(tot_res)
tot_acc = np.concatenate(tot_acc)
tot_loss = np.concatenate(tot_loss)

print(f"Evaluation: {np.mean(tot_res[:,1])*100}%")
print(f"Accuracy: {np.mean(tot_acc)*100}%")
print(f"Loss: {np.mean(tot_res[:,0])*100}%")


Evaluation: 79.56640323003134%
Accuracy: 80.94775970093906%
Loss: 63.53173702955246%


In [50]:
tot_loss

array([[0.75614339, 0.65435392, 0.63145953, 0.61665827, 0.60239607,
        0.59254134, 0.58622319, 0.5772047 , 0.57176536, 0.56493288,
        0.55884326, 0.55213171, 0.54851186, 0.5422467 , 0.53837776,
        0.53398985, 0.53016746, 0.52553058, 0.52117985, 0.51914328,
        0.51350856, 0.50858009, 0.50235802, 0.5010227 , 0.49814254,
        0.49248004, 0.48707789, 0.48182166, 0.47663063, 0.47622734,
        0.47290981, 0.46919096],
       [0.76381189, 0.66896796, 0.64357871, 0.62668639, 0.61258781,
        0.60255265, 0.59373611, 0.58503115, 0.57967776, 0.57218271,
        0.56759501, 0.56094217, 0.55642712, 0.55350399, 0.54746884,
        0.54524457, 0.53760231, 0.53334343, 0.53070319, 0.52803856,
        0.52146858, 0.519777  , 0.51445985, 0.51265746, 0.5074895 ,
        0.50312352, 0.50126779, 0.49547672, 0.49367639, 0.49053273,
        0.48666561, 0.48179689],
       [0.7372759 , 0.63981593, 0.61673158, 0.60458446, 0.59282297,
        0.5814684 , 0.57517779, 0.56678975, 0.5633

In [52]:
np.mean(tot_loss[:,31])

0.4688990960518519

In [ ]:
print(f"Result: {np.mean(tot_res_copy[:,0])*100}%")

In [ ]:
metrics_values = evaluation_results[1, :]

In [ ]:
tot_res_copy = tot_res

In [ ]:
tot_res_copy = np.concatenate(tot_res_copy)

In [ ]:
tot_res_copy

In [ ]:
tot_loss[:,31]

In [ ]:
# epochs = range(len(acc))  # Number of epochs

In [ ]:
np.mean(accuracy[0][:])
avg_accuracy = [np.mean(sublist) for sublist in accuracy]
avg_accuracy

In [ ]:
np.mean(avg_accuracy)

In [ ]:
np.mean(loss)

In [ ]:
# # Plotting training and validation accuracy
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(epochs, acc, label='Training Accuracy')
# # plt.plot(epochs, val_acc, label='Validation Accuracy')
# plt.title('Training Accuracy')
# plt.legend()
# 
# # Plotting training and validation loss
# plt.subplot(1, 2, 2)
# plt.plot(epochs, loss, label='Training Loss')
# # plt.plot(epochs, val_loss, label='Validation Loss')
# plt.title('Training Loss')
# plt.legend()
# 
# plt.show()

In [ ]:
# def build_lstm():
#     lstm_model = Sequential()
#     lstm_model.add(TimeDistributed(cnn_model, input_shape=(35, input_shape[0], input_shape[1])))  # Assuming input_shape is (20, 6)
#     lstm_model.add(LSTM(64, activation='tanh', recurrent_activation='hard_sigmoid', return_sequences=True))
#     lstm_model.add(LSTM(64, activation='tanh', recurrent_activation='hard_sigmoid'))
#     lstm_model.add(Dropout(0.5))
#     lstm_model.add(Dense(1, activation='sigmoid'))  # Binary classification
# 
#     lstm_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
#     return lstm_model
